In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os

In [2]:
import pandas as pd

In [23]:
import random

### Hyper params

In [114]:
csvFile = pd.read_csv('../dataset/data_cog.csv')
data = csvFile.values
Xdata = data[:, :-5] 
Xdata = Xdata / np.linalg.norm(Xdata, 2)
ydata = data[:, -4:-1] 
ydata = ydata / np.linalg.norm(ydata, 2)

In [115]:
(n, d) = Xdata.shape

In [116]:
n_train = n // 2
n_val = n // 10

X_train = Xdata[:n_train]
X_val   = Xdata[n_train:n_train+n_val]
X_test  = Xdata[n_train+n_val:]

y_train = ydata[:n_train]
y_val   = ydata[n_train:n_train+n_val]
y_test  = ydata[n_train+n_val:]

In [117]:
mb_size = 
Z_dim = d
X_dim = ydata.shape[1]
y_dim = 1
h_dim = 20

In [118]:
c = 0
lr = 1e-3

In [119]:
def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape=size, stddev=xavier_stddev)

### Geneartor

In [120]:
Z = tf.placeholder(tf.float32, shape=[None, Z_dim])

G_W1 = tf.Variable(xavier_init([Z_dim, h_dim]))
G_b1 = tf.Variable(tf.zeros(shape=[h_dim]))

G_W2 = tf.Variable(xavier_init([h_dim, X_dim]))
G_b2 = tf.Variable(tf.zeros(shape=[X_dim]))

theta_G = [G_W1, G_W2, G_b1, G_b2]

In [121]:
def generator(z):
    G_h1 = tf.nn.relu(tf.matmul(z, G_W1) + G_b1)
    G_log_prob = tf.matmul(G_h1, G_W2) + G_b2
    G_prob = tf.nn.relu(G_log_prob)

    return G_prob

### Discriminator

In [122]:
X = tf.placeholder(tf.float32, shape=[None, X_dim])
D_W1 = tf.Variable(xavier_init([X_dim, h_dim]))
D_b1 = tf.Variable(tf.zeros(shape=[h_dim]))

D_W2 = tf.Variable(xavier_init([h_dim, 1]))
D_b2 = tf.Variable(tf.zeros(shape=[1]))

theta_D = [D_W1, D_W2, D_b1, D_b2]

In [123]:
def discriminator(x):
    D_h1 = tf.nn.relu(tf.matmul(x, D_W1) + D_b1)
    D_logit = tf.matmul(D_h1, D_W2) + D_b2
    D_prob = tf.nn.sigmoid(D_logit)

    return D_prob, D_logit

###  Training

In [124]:
def sample_Z(m, n):
    return np.random.uniform(-1., 1., size=[m, n])

In [125]:
def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

    return fig

#### Generate data

In [126]:
G_sample = generator(Z)

In [127]:
D_real, D_logit_real = discriminator(X)

In [128]:
D_fake, D_logit_fake = discriminator(G_sample)

#### Loss

In [129]:
D_loss_real = tf.reduce_mean(
    tf.nn.sigmoid_cross_entropy_with_logits(
        logits=D_logit_real, labels=tf.ones_like(D_logit_real)))
D_loss_fake = tf.reduce_mean(
    tf.nn.sigmoid_cross_entropy_with_logits(
        logits=D_logit_fake, labels=tf.zeros_like(D_logit_fake)))
D_loss = D_loss_real + D_loss_fake

G_loss = tf.reduce_mean(
    tf.nn.sigmoid_cross_entropy_with_logits(
        logits=D_logit_fake, labels=tf.ones_like(D_logit_fake)))

#### Optimize

In [130]:
D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list=theta_D)
G_solver = tf.train.AdamOptimizer().minimize(G_loss, var_list=theta_G)

#### Running

In [131]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [132]:
i = 0
for it in range(10000):
    wSet = random.sample(range(n), mb_size)
    y_mb = ydata[wSet, :]
    _, D_loss_curr = sess.run([D_solver, D_loss], 
                  feed_dict={X: y_mb, Z: Xdata[wSet, :]})
    _, G_loss_curr = sess.run([G_solver, G_loss], 
                  feed_dict={Z: Xdata[wSet, :]})

    if it % 100 == 0:
        print('Iter: {}'.format(it))
        print('D loss: {:.4}'. format(D_loss_curr))
        print('G_loss: {:.4}'.format(G_loss_curr))
        print()

Iter: 0
D loss: 1.388
G_loss: 0.6925

Iter: 100
D loss: 1.295
G_loss: 0.736

Iter: 200
D loss: 1.236
G_loss: 0.7753

Iter: 300
D loss: 1.146
G_loss: 0.8247

Iter: 400
D loss: 1.044
G_loss: 0.901

Iter: 500
D loss: 0.9468
G_loss: 1.003

Iter: 600
D loss: 0.8342
G_loss: 1.119

Iter: 700
D loss: 0.6985
G_loss: 1.237

Iter: 800
D loss: 0.6163
G_loss: 1.324

Iter: 900
D loss: 0.5801
G_loss: 1.454

Iter: 1000
D loss: 0.4566
G_loss: 1.616

Iter: 1100
D loss: 0.3125
G_loss: 1.76

Iter: 1200
D loss: 0.4072
G_loss: 1.919

Iter: 1300
D loss: 0.268
G_loss: 2.068

Iter: 1400
D loss: 0.2814
G_loss: 2.157

Iter: 1500
D loss: 0.2492
G_loss: 2.29

Iter: 1600
D loss: 0.2544
G_loss: 2.452

Iter: 1700
D loss: 0.1572
G_loss: 2.548

Iter: 1800
D loss: 0.1397
G_loss: 2.538

Iter: 1900
D loss: 0.1153
G_loss: 2.711

Iter: 2000
D loss: 0.108
G_loss: 2.875

Iter: 2100
D loss: 0.06953
G_loss: 3.045

Iter: 2200
D loss: 0.09538
G_loss: 3.015

Iter: 2300
D loss: 0.1064
G_loss: 3.22

Iter: 2400
D loss: 0.07705
G_loss

### Evaluation

In [112]:
pred_train = sess.run(generator(Z), feed_dict={Z: X_train})

In [113]:
pred_train - y_train

array([[ -785.33 ,  -357.47 ,   -71.218],
       [ -660.46 ,  -176.41 ,   -32.398],
       [ -602.03 ,  -346.18 ,   -53.077],
       [ -744.64 ,  -190.4  ,   -35.72 ],
       [-1019.   ,  -313.032,   -48.002],
       [ -877.8  ,  -480.89 ,   -60.402],
       [ -958.32 ,  -335.36 ,   -55.685],
       [ -588.27 ,  -304.72 ,   -44.938],
       [ -402.79 ,  -352.21 ,   -59.156],
       [ -884.57 ,  -394.56 ,   -64.998],
       [ -536.   ,  -578.   ,   -62.   ],
       [ -849.52 ,  -335.36 ,   -51.296],
       [ -510.2  ,  -346.69 ,   -57.175],
       [ -604.77 ,  -201.52 ,   -34.264],
       [ -901.87 ,  -306.49 ,   -50.725],
       [ -786.02 ,  -354.84 ,   -63.653],
       [ -749.59 ,  -241.02 ,   -39.433],
       [ -699.   ,  -270.023,   -43.907],
       [ -357.26 ,  -559.41 ,   -66.142],
       [-1006.2  ,  -250.76 ,   -37.874],
       [ -270.25 ,  -554.23 ,   -83.881],
       [ -845.   ,  -296.   ,   -33.   ],
       [ -621.   ,  -199.162,   -35.513],
       [ -988.09 ,  -303.47 ,   -4